In [1]:
from random import choice
from enbios2.base.experiment import Experiment
import bw2data

from enbios2.models.experiment_models import ExperimentData

In [2]:
bw2data.projects.set_current("ecoinvent")
exp = Experiment.create("ecoinvent")

2023-07-05 16:07:07,850 - enbios2.base.experiment - DEBUG - activity list


In [3]:
list(bw2data.databases)

['biosphere3',
 'apos_3.9.1_default',
 'consequential_3.9.1_default',
 'cutoff_3.9.1_default',
 'cutoff_3.9_default']

In [11]:
database_name = 'cutoff_3.9.1_default'
db = bw2data.Database(database_name)
activities = [db.random() for _ in range(10)]
activities

['market for waste polyethylene terephthalate' (kilogram, SI, None),
 'market group for natural gas, high pressure' (cubic meter, GLO, None),
 'market for paper, woodcontaining, lightweight coated' (kilogram, RER, None),
 'electricity production, hydro, pumped storage' (kilowatt hour, CN-JS, None),
 'electricity production, hydro, run-of-river' (kilowatt hour, IN-JH, None),
 'fibre production, cotton, ginning' (kilogram, IN, None),
 'treatment of spent antifreezer liquid, hazardous waste incineration' (kilogram, RoW, None),
 'pyridine-compound production' (kilogram, RER, None),
 'planing, lath, softwood, u=20%' (cubic meter, Europe without Switzerland, None),
 'generator production, 200kW electrical' (unit, RER, None)]

In [21]:
experiment_activities = [
    {
        "id": {
            "code": act["code"],
            "name": act["name"],
        },
        "output": [act["unit"], 1]
    }
    for act in activities
]

In [13]:
experiment_activities

[{'id': {'code': '18277df8a10170108ac21b60acb13c16'},
  'output': ['kilogram', 1]},
 {'id': {'code': 'a41a1c1d58c1e6160f6c6516f7f4bff9'},
  'output': ['cubic meter', 1]},
 {'id': {'code': 'd7499ddf8138ff5183e068df563c0654'},
  'output': ['kilogram', 1]},
 {'id': {'code': 'b8a74c3cf4075b7bbb23b6b50d9f8676'},
  'output': ['kilowatt hour', 1]},
 {'id': {'code': 'ba7d962ab827002fbc8df6ff0086f318'},
  'output': ['kilowatt hour', 1]},
 {'id': {'code': '807ecabdf7a9f10e6801fb0723e86f57'},
  'output': ['kilogram', 1]},
 {'id': {'code': 'a8bd1a2516a229ff785b789f89f10cd2'},
  'output': ['kilogram', 1]},
 {'id': {'code': 'd1588ab1309a995d9ad42ce62655efb9'},
  'output': ['kilogram', 1]},
 {'id': {'code': '1588fc543ab66e47895120ed281029f2'},
  'output': ['cubic meter', 1]},
 {'id': {'code': '4b29bed545b4cd0b308c4514d31f4ba9'}, 'output': ['unit', 1]}]

In [14]:
all_methods = list(bw2data.methods)
methods = [choice(all_methods) for _ in range(3)]
methods

[('EF v3.1 EN15804', 'ozone depletion', 'ozone depletion potential (ODP)'),
 ('ReCiPe 2016 v1.03, endpoint (E)',
  'total: natural resources',
  'natural resources'),
 ('ReCiPe 2016 v1.03, endpoint (H)',
  'ecosystem quality',
  'photochemical oxidant formation: terrestrial ecosystems')]

In [15]:
experiment_methods = [
    {
        "id": method
    }
    for method in methods
]

In [16]:
exp_data = ExperimentData(
    bw_project="ecoinvent",
    activities=experiment_activities,
    methods=experiment_methods
)

In [17]:
exp: Experiment = Experiment(exp_data)

2023-07-05 16:19:44,695 - enbios2.base.experiment - DEBUG - activity list


In [18]:
results = exp.run()

2023-07-05 16:19:46,682 - enbios2.base.experiment - INFO - Running scenario 'default scenario'
2023-07-05 16:21:30,428 - enbios2.base.experiment - WARNING - Cannot aggregate output to parent: root. From earlier children the base unit is 1.0 kilogram and from market group for natural gas, high pressure it is meter ** 3. Cannot convert from 'kilogram' ([mass]) to 'meter ** 3' ([length] ** 3)
2023-07-05 16:21:30,431 - enbios2.base.experiment - WARNING - Cannot aggregate output to parent: root. From earlier children the base unit is 2.0 kilogram and from electricity production, hydro, pumped storage it is kilowatt_hour. Cannot convert from 'kilogram' ([mass]) to 'kilowatt_hour' ([length] ** 2 * [mass] / [time] ** 2)
2023-07-05 16:21:30,432 - enbios2.base.experiment - WARNING - Cannot aggregate output to parent: root. From earlier children the base unit is 2.0 kilogram and from electricity production, hydro, run-of-river it is kilowatt_hour. Cannot convert from 'kilogram' ([mass]) to 'kilow

In [20]:
results

{'default scenario': {'name': 'root',
  'children': [{'name': 'market for waste polyethylene terephthalate',
    'children': [],
    'data': ScenarioResultNodeData(output=('kilogram', 1.0), results={'EF v3.1 EN15804_ozone depletion_ozone depletion potential (ODP)': -6.968850056331269e-10, 'ReCiPe 2016 v1.03, endpoint (E)_total: natural resources_natural resources': -0.0036601317945314927, 'ReCiPe 2016 v1.03, endpoint (H)_ecosystem quality_photochemical oxidant formation: terrestrial ecosystems': -6.448358281675913e-11})},
   {'name': 'market group for natural gas, high pressure',
    'children': [],
    'data': ScenarioResultNodeData(output=('cubic_meter', 1.0), results={'EF v3.1 EN15804_ozone depletion_ozone depletion potential (ODP)': 6.015459474346291e-08, 'ReCiPe 2016 v1.03, endpoint (E)_total: natural resources_natural resources': 0.340832970055725, 'ReCiPe 2016 v1.03, endpoint (H)_ecosystem quality_photochemical oxidant formation: terrestrial ecosystems': 2.3076906566011318e-10})

In [ ]:
# SCENARIOS NEED SOME REPAIR!

In [32]:
exp_data.scenarios = []

for _ in range(3):
    random_activity = choice(list(exp.activitiesMap.keys()))
    # print(random_activity)
    exp_data.scenarios.append(
        {
            random_activity: [exp.activitiesMap[random_activity].output.unit, 3]
        }
    )

exp_data.scenarios

[{'pyridine-compound production': ['kilogram', 3]},
 {'electricity production, hydro, run-of-river': ['kilowatt hour', 3]},
 {'electricity production, hydro, pumped storage': ['kilowatt hour', 3]}]

In [34]:
exp: Experiment = Experiment(exp_data)
results = exp.run()

2023-07-05 16:32:03,819 - enbios2.base.experiment - DEBUG - activity list


AttributeError: 'dict' object has no attribute 'alias'

In [27]:
# exp.scenarios

[]